In [1]:
import utilities
import constants

In [2]:
utilities.prepare_env()

### Trends from Gemini with Google Search
Top 10 for the last 7 days

In [14]:
gemini_trends = utilities.generate_gemini_trends()
gemini_trends.head(3)

,trend,description,ranking,date,category,source
0,Fiestas Patrias,Búsqueda de preparativos para las Fiestas Patr...,1,2025-09-09,holiday,gemini_search
1,Entradas a conciertos,Alta demanda y búsqueda de entradas para próxi...,2,2025-09-09,music,gemini_search
2,Moda Primavera-Verano,Lanzamiento y búsqueda de las nuevas coleccion...,3,2025-09-08,fashion,gemini_search


In [15]:
utilities.save_bq_table(constants.table_name_gemini_search, gemini_trends)

### Youtube popular videos
For chile in spanish language

In [9]:
popular_videos = utilities.get_popular_videos(constants.YOUTUBE_API_KEY, region_code='CL', max_results=3)
popular_videos

,title,url,views,likes,summary,keywords,category,source,date
0,Beéle - si te pillara (Letra/Lyrics),https://www.youtube.com/watch?v=VEed1BGRFKY,123656,338,El video es una presentación lírica de la canc...,"música, desamor, celos, infidelidad, Beéle, mú...","Música, Artistas",youtube,2025-09-10
1,Hazbin Hotel - Season 2 Teaser Trailer | Prime...,https://www.youtube.com/watch?v=fg2JZQIj7Bg,658022,103493,Tráiler de la nueva temporada de la serie de a...,"Hazbin Hotel, animación para adultos, Vivienne...","series de televisión, animación, artistas",youtube,2025-09-10
2,Consegui TODOS los CRAFTEOS de BRAINROTS SECRE...,https://www.youtube.com/watch?v=K0_TGVTb8c8,997639,7711,El video muestra a un jugador en el videojuego...,"videojuegos, Roblox, Steal a Brainrot, crafteo...",Videojuegos,youtube,2025-09-10


In [13]:
utilities.save_bq_table(constants.table_name_youtube, popular_videos)

### Google Trends in BigQuery

In [7]:
df_bq_trends = utilities.get_bq_trends()
df_bq_trends

,region_name,term,score,week,rank,source,categories
0,Antofagasta Region,concepción cobreloa,100,2025-09-07,5,google_trends,"Sports, Football"
1,Antofagasta Region,concepción cobreloa,100,2025-09-07,5,google_trends,"Sports, Football"
2,Antofagasta Region,hora oficial chile,100,2025-09-07,4,google_trends,General Interest
3,Antofagasta Region,video de isabella ladera y beele,100,2025-09-07,4,google_trends,"Entertainment, Celebrities, Social Media"
4,Antofagasta Region,turquía españa,100,2025-09-07,3,google_trends,"Sports, Football, Event"
...,...,...,...,...,...,...,...
145,XI Región,cobresal palestino,100,2025-09-07,3,google_trends,"Sports, Football"
146,XI Región,deportes limache la serena,29,2025-09-07,2,google_trends,"Sports, Football"
147,XI Región,armenia portugal,<NA>,2025-09-07,2,google_trends,"Sports, Football, Event"
148,XI Región,turquía españa,42,2025-09-07,1,google_trends,"Sports, Football, Event"


In [8]:
utilities.save_bq_table(constants.table_name_google_trends, df_bq_trends)

## Analyze trends together

In [7]:
SQL = f"""
    SELECT source, date(week) date, term, categories FROM `{constants.project_id}.{constants.dataset_name}.{constants.table_name_google_trends}`
    UNION ALL
    SELECT source, date(date) date, title, category categories FROM `{constants.project_id}.{constants.dataset_name}.{constants.table_name_youtube}`
    UNION ALL
    SELECT source, date(date) date, trend, category categories FROM `{constants.project_id}.{constants.dataset_name}.{constants.table_name_gemini_search}`
"""

In [8]:
trends = utilities.run_bq_query_df(SQL)
trends

,source,date,term,categories
0,gemini_search,2025-09-09,Fiestas Patrias,holiday
1,gemini_search,2025-09-09,Entradas a conciertos,music
2,gemini_search,2025-09-08,Moda Primavera-Verano,fashion
3,gemini_search,2025-09-08,Estrenos de Series,entertainment
4,gemini_search,2025-09-07,Ofertas de Viaje,travel
...,...,...,...,...
168,google_trends,2025-09-07,concepción cobreloa,"Sports, Football"
169,google_trends,2025-09-07,concepción cobreloa,"Sports, Football"
170,youtube,2025-09-10,Beéle - si te pillara (Letra/Lyrics),"Música, Artistas"
171,youtube,2025-09-10,Hazbin Hotel - Season 2 Teaser Trailer | Prime...,"series de televisión, animación, artistas"
